In [1]:
# This chunk of code make it possible to use src functions
import sys
import os
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from DSProject import DSProject

dsp = DSProject()

# Read in files and clean up

In [2]:
LA = dsp.readPreprocessedData('ListAction')

AttributeError: 'DSProject' object has no attribute 'readPreprocessedData'

In [3]:
db = dsp.readPreprocessedData('DB_SOC')

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [10]:
LA['date'] = pd.datetime(LA['FDTACTIONDATE'], '%Y-%m-%d %H:%M:%S')

TypeError: cannot convert the series to <class 'int'>

In [9]:
LA['FDTACTIONDATE']

0          2015-10-05 08:50:22
1          2015-10-05 08:50:38
2          2015-10-05 09:38:27
3          2015-10-05 09:38:33
4          2015-10-05 09:39:32
5          2015-10-05 08:57:03
6          2015-10-05 08:58:00
7          2015-10-05 08:58:32
8          2015-10-05 09:39:41
9          2015-10-05 09:39:46
10         2015-10-05 09:40:13
11         2015-10-05 09:17:08
12         2015-10-05 09:16:37
13         2015-10-05 08:35:04
14         2015-10-05 09:20:44
15         2015-10-05 09:22:44
16         2015-10-05 09:17:21
17         2015-10-05 09:20:18
18         2015-10-05 09:20:27
19         2015-10-05 09:20:27
20         2015-10-05 09:20:27
21         2015-10-05 08:57:39
22         2015-10-05 08:57:39
23         2015-10-05 08:57:39
24         2015-10-05 08:57:39
25         2015-10-05 09:43:43
26         2015-10-05 09:43:43
27         2015-10-05 09:23:36
28         2015-10-05 09:23:36
29         2015-10-05 09:24:09
                  ...         
2270550    2016-04-01 17:06:32
2270551 

In [4]:
from datetime import datetime
import pandas as pd

LA['FileDate_day'] = pd.DatetimeIndex(LA['FDTACTIONDATE']).day
LA['FileDate_month'] = pd.DatetimeIndex(LA['FDTACTIONDATE']).month
LA['FileDate_year'] = pd.DatetimeIndex(LA['FDTACTIONDATE']).year
LA['Hours'] = pd.DatetimeIndex(LA['FDTACTIONDATE']).hour

merge = pd.merge(LA, db, how = 'left', on = ['HASH_AR_ID','HASH_LPM_CST_ID','FileDate_month','FileDate_year'])

KeyError: 'FileDate_month'

In [23]:
merge = merge.drop(['FSZCOLLECTORCODE', 'FSZCOLLECTORSUP'], axis = 1)

In [24]:
#drop all with NaN

merge = merge.dropna(axis = 0, how='any')



## Create new features

In [25]:
merge['Days_past'] = (merge['FileDate_day'] - merge['BILLCycle'])
merge.head()

merge['Days_past'] = merge['Days_past'].apply(lambda x: x+30 if x < 0 else x)


In [26]:
b1 = merge.loc[merge['Range_day'] == 1]
b2 = merge.loc[merge['Range_day'] == 2]
b3 = merge.loc[merge['Range_day'] == 3]

b_list = [b1,b2,b3]

# Feature Selection

In [27]:
category_columns = [
       'FDTACTIONDATE', 'FDTRESULTDATE', 'Hours',
       'FISRNO', 'ACTION_DESC', 'ACTIONCOLL', 'StampDate', 'HASH_LPM_CST_ID',
       'HASH_AR_ID', 'FileDate_month', 'FileDate_year', 'Unnamed: 0_y',
       'Month', 'FileDate', 'PortFolio', 'ProductName',
       'BILLCycle', 'Group', 'Range_day', 'Card Type New', 'New_Handle']
numeric_columns = ['OSAMT', 'OSPRINCIPLE', 'Days_past']

In [28]:
from sklearn import preprocessing
labenc = preprocessing.LabelEncoder()

for b in b_list:
    for column in category_columns:
        b[column] = labenc.fit_transform(b[column].tolist())

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
y_onehot_b1 = pd.get_dummies(b1['FResultcategory'])
y_onehot_b2 = pd.get_dummies(b2['FResultcategory'])
y_onehot_b3 = pd.get_dummies(b3['FResultcategory'])

y_list = [y_onehot_b1,y_onehot_b2,y_onehot_b3]



data_num = [(b1[numeric_columns],y_onehot_b1),(b2[numeric_columns],y_onehot_b2),(b3[numeric_columns],y_onehot_b3)]

data_nom = [(b1[category_columns],y_onehot_b1),(b2[category_columns],y_onehot_b2),(b3[category_columns],y_onehot_b3)]





In [30]:
from sklearn.feature_selection import chi2, f_classif
import numpy as np

def test_feature(x, y, func=f_classif):
    f_group1 = func(x, y)
    f_group1_table = pd.DataFrame(np.array([list(x.columns), f_group1[0], f_group1[1]]).T,
                                  columns=['column_name', 'f_value', 'p_value'])
    f_group1_table['f_value'] = f_group1_table['f_value'].astype(np.float64)
    f_group1_table['p_value'] = f_group1_table['p_value'].astype(np.float64)
    col_to_drop = f_group1_table.loc[f_group1_table['p_value'] > 0.001]['column_name'].tolist()
    return f_group1_table, col_to_drop

In [31]:
df_numeric = []

for d in data_num:
    b,y = d[0], d[1]
    result_table, col_to_drop = test_feature(b, y['PTP'])
    df_numeric1 = b.drop(col_to_drop, axis=1)
    df_numeric.append(df_numeric1)

In [32]:
df_nominal = []

for d in data_nom:

    
    b,y = d[0], d[1]
    result_table, col_to_drop = test_feature(b, y['PTP'],chi2)
    df_cat1 = b.drop(col_to_drop, axis=1)
    df_nominal.append(df_cat1)


In [33]:
df_nominal[0] = pd.get_dummies(df_nominal[0], columns = ['Hours','ACTIONCOLL', 'Group'])
df_nominal[1] = pd.get_dummies(df_nominal[1], columns = ['Hours','ACTIONCOLL', 'Group'])
df_nominal[2] = pd.get_dummies(df_nominal[2], columns = ['Hours','ACTIONCOLL', 'Group'])

In [34]:
x = []

for i in range(3):
    x.append(pd.concat([df_nominal[i],df_numeric[i]],axis = 1))


In [35]:
col = ['OSAMT', 'FISRNO','OSPRINCIPLE','ACTION_DESC', 'FDTACTIONDATE', 'HASH_LPM_CST_ID', 'HASH_AR_ID',
       'FDTRESULTDATE', 'Unnamed: 0_y', 'ProductName', 'New_Handle']

for i in range(3):
    x[i] = x[i].drop(col, axis = 1)

# Models

In [36]:
x[0].head()

,StampDate,FileDate_month,FileDate_year,Month,FileDate,PortFolio,BILLCycle,Range_day,Card Type New,Hours_0,...,ACTIONCOLL_398,ACTIONCOLL_399,ACTIONCOLL_400,ACTIONCOLL_401,Group_0,Group_1,Group_2,Group_3,Group_4,Days_past
1,3,9,0,0,1,0,3,0,0,1,...,0,0,0,0,0,0,1,0,0,15.0
11,3,9,0,0,2,0,4,0,5,0,...,0,0,0,0,0,0,1,0,0,10.0
33,3,9,0,0,0,0,2,0,5,0,...,0,0,0,0,0,0,0,0,1,18.0
42,3,9,0,0,2,0,4,0,6,0,...,0,0,0,0,0,0,1,0,0,10.0
52,3,9,0,0,1,0,3,0,6,1,...,0,0,0,0,0,0,1,0,0,15.0


delete stampdate
time delta (stampdate - file date)
add weekday (MTWTF)


In [37]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report

def test_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)
    yhat = model.predict(x_test)
    result = classification_report(y_test, yhat)
    print(result)
    return model

mod = []


for i in range(3):
    
    print ("Bucket " + (str)(i+1))
    print ("Spliting training/testing set")
    
    x_train, x_test, y_train, y_test = train_test_split(x[i], y_list[i]['PTP'], test_size=0.2)
    
    print ("Building Model")

    estim = {'LR': LR(solver='sag', C=0.1, random_state=42, n_jobs=-1), 'RFC': RFC(n_estimators=50, n_jobs=-1, oob_score=True, min_samples_leaf=10, random_state=42)}
    mod1 = {}
    for key, value in estim.items():
        
        print ("Testing Model for Bucket " + (str)(i+1) + " using " + key )
        
        mod1[key] = test_model(value, x_train, y_train, x_test, y_test)
        
    mod.append(mod1)

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Bucket 1
Spliting training/testing set
Building Model
Testing Model for Bucket 1 using LR


/anaconda/lib/python3.6/site-packages/sklearn/linear_model/sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


             precision    recall  f1-score   support

          0       0.70      0.84      0.76     99981
          1       0.61      0.43      0.50     61220

avg / total       0.67      0.68      0.67    161201

Testing Model for Bucket 1 using RFC
             precision    recall  f1-score   support

          0       0.72      0.85      0.78     99981
          1       0.65      0.45      0.53     61220

avg / total       0.69      0.70      0.69    161201

Bucket 2
Spliting training/testing set
Building Model
Testing Model for Bucket 2 using LR


/anaconda/lib/python3.6/site-packages/sklearn/linear_model/sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


             precision    recall  f1-score   support

          0       0.80      0.98      0.88     43373
          1       0.59      0.12      0.20     12000

avg / total       0.76      0.79      0.73     55373

Testing Model for Bucket 2 using RFC
             precision    recall  f1-score   support

          0       0.80      0.98      0.88     43373
          1       0.64      0.10      0.18     12000

avg / total       0.76      0.79      0.73     55373

Bucket 3
Spliting training/testing set
Building Model
Testing Model for Bucket 3 using LR


/anaconda/lib/python3.6/site-packages/sklearn/linear_model/sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


             precision    recall  f1-score   support

          0       0.87      1.00      0.93     30822
          1       0.60      0.01      0.01      4819

avg / total       0.83      0.87      0.80     35641

Testing Model for Bucket 3 using RFC
             precision    recall  f1-score   support

          0       0.87      1.00      0.93     30822
          1       0.66      0.02      0.05      4819

avg / total       0.84      0.87      0.81     35641



We analyze data using 2 models: Logistic Regression, and Random Forest. The latter gives better accuracy for PTP prediction. Given the data from List Action and DB_SOC, the model can predict PTP with 74% accuracy, and predict non-PTP with 89% accuracy.

### Features' Importance

In [38]:
for i in range(3):

    summary = {'Features':x[i].columns, 'Importance':mod[i]['RFC'].feature_importances_}
    summary = pd.DataFrame(data = summary)
    print ("Summary for Bucket "+ (str)(i+1))
    print (summary.sort('Importance', ascending = 0))

Summary for Bucket 1
           Features  Importance
429       Days_past    0.103984
428         Group_4    0.098051
426         Group_2    0.096763
0         StampDate    0.064583
8     Card Type New    0.047366
10          Hours_1    0.047221
4          FileDate    0.043955
9           Hours_0    0.042589
3             Month    0.027342
17          Hours_8    0.021533
6         BILLCycle    0.020214
364  ACTIONCOLL_342    0.018573
369  ACTIONCOLL_347    0.018345
1    FileDate_month    0.018325
19         Hours_10    0.017487
16          Hours_7    0.017298
366  ACTIONCOLL_344    0.016871
367  ACTIONCOLL_345    0.015985
11          Hours_2    0.014488
18          Hours_9    0.011756
368  ACTIONCOLL_346    0.010700
365  ACTIONCOLL_343    0.009982
371  ACTIONCOLL_349    0.007768
12          Hours_3    0.007683
354  ACTIONCOLL_332    0.007634
352  ACTIONCOLL_330    0.007586
15          Hours_6    0.007348
374  ACTIONCOLL_352    0.007326
370  ACTIONCOLL_348    0.006603
358  ACTIONCOLL_336

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Summary for Bucket 3
           Features  Importance
335         Group_4    0.162241
333         Group_2    0.120348
7     Card Type New    0.113158
0         StampDate    0.048750
336       Days_past    0.046624
317  ACTIONCOLL_295    0.045727
4          FileDate    0.035046
45    ACTIONCOLL_23    0.030129
312  ACTIONCOLL_290    0.026481
5         BILLCycle    0.022738
3             Month    0.022664
1    FileDate_month    0.021644
9           Hours_1    0.019115
311  ACTIONCOLL_289    0.017934
315  ACTIONCOLL_293    0.015164
313  ACTIONCOLL_291    0.010915
298  ACTIONCOLL_276    0.009305
334         Group_3    0.009182
292  ACTIONCOLL_270    0.008714
10          Hours_2    0.007179
12          Hours_4    0.006934
11          Hours_3    0.006326
89    ACTIONCOLL_67    0.005725
296  ACTIONCOLL_274    0.005686
316  ACTIONCOLL_294    0.005629
300  ACTIONCOLL_278    0.005629
303  ACTIONCOLL_281    0.005567
293  ACTIONCOLL_271    0.005531
297  ACTIONCOLL_275    0.005409
17          Hours_9

This table shows the importance of each features. We can see that the agents affect the result more than other factors.


For Hours features, Hour_0 = 7am, Hour_1 = 8am, Hour_2 = 9am, ..., Hour_13 = 8pm


For Group features, Group_0 = self_cured, Group_2 = Action_Save, Group_4 = Action_Roll